In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [ ]:
with open(r"C:\Users\hunte\Documents\NNS-DDA10\python\data\census_api_key.json") as fi:
    credentials = json.load(fi)

In [ ]:
api_key = credentials['api_key']

In [ ]:
#bring in median income

In [ ]:
endpoint = 'https://api.census.gov/data/timeseries/poverty/saipe?get=NAME,SAEMHI_PT&for=county:*&in=state:47&YEAR=2020&key={0}'.format(api_key)

response = requests.get(endpoint)

In [ ]:
response.text

In [ ]:
res = response.json()

In [ ]:
income = pd.DataFrame(res)

In [ ]:
income

In [ ]:
#bring in population

In [ ]:
endpoint2 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P1_001N&for=county:*&in=state:47&key={0}'.format(api_key)

response2 = requests.get(endpoint2)

In [ ]:
pop = pd.DataFrame(response2.json())
pop

In [ ]:
# Here we pull in population for each county for 65+
# This will lead to many joins to get a generaltable for all populations 65+

In [ ]:
endpoint3 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_020N&for=county:*&in=state:47&key={0}'.format(api_key)

response3 = requests.get(endpoint3)

In [ ]:
male_65_66 = pd.DataFrame(response3.json())

In [ ]:
headers = male_65_66.iloc[0]
male_65_66 = male_65_66[1:]
male_65_66.columns = headers

In [ ]:
male_65_66 = male_65_66.rename(columns = {'NAME' : 'county_name',
                                          'P12_020N' : 'male pop 65-66'})

In [ ]:
endpoint4 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_021N&for=county:*&in=state:47&key={0}'.format(api_key)

response4 = requests.get(endpoint4)

In [ ]:
male_67_69 = pd.DataFrame(response4.json())

In [ ]:
headers = male_67_69.iloc[0]
male_67_69 = male_67_69[1:]
male_67_69.columns = headers

In [ ]:
male_67_69 = male_67_69.rename(columns = {'NAME' : 'county_name',
                                          'P12_021N' : 'male pop 67_69'})

In [ ]:
endpoint5 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_022N&for=county:*&in=state:47&key={0}'.format(api_key)

response5 = requests.get(endpoint5)

In [ ]:
male_70_74 = pd.DataFrame(response5.json())

In [ ]:
headers = male_70_74.iloc[0]
male_70_74 = male_70_74[1:]
male_70_74.columns = headers

In [ ]:
male_70_74 = male_70_74.rename(columns = {'NAME' : 'county_name',
                                          'P12_022N' : 'male pop 70-74'})

In [ ]:
endpoint6 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_023N&for=county:*&in=state:47&key={0}'.format(api_key)

response6 = requests.get(endpoint6)

In [ ]:
male_75_79 = pd.DataFrame(response6.json())

In [ ]:
headers = male_75_79.iloc[0]
male_75_79 = male_75_79[1:]
male_75_79.columns = headers

In [ ]:
male_75_79 = male_75_79.rename(columns = {'NAME' : 'county_name',
                                          'P12_023N' : 'male pop 75-79'})

In [ ]:
endpoint7 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_024N&for=county:*&in=state:47&key={0}'.format(api_key)

response7 = requests.get(endpoint7)

In [ ]:
male_80_84 = pd.DataFrame(response7.json())

In [ ]:
headers = male_80_84.iloc[0]
male_80_84 = male_80_84[1:]
male_80_84.columns = headers

In [ ]:
male_80_84 = male_80_84.rename(columns = {'NAME' : 'county_name',
                                          'P12_024N' : 'male pop 80-84'})

In [ ]:
endpoint8 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_025N&for=county:*&in=state:47&key={0}'.format(api_key)

response8 = requests.get(endpoint8)

In [ ]:
male_80_plus = pd.DataFrame(response8.json())

In [ ]:
headers = male_80_plus.iloc[0]
male_80_plus = male_80_plus[1:]
male_80_plus.columns = headers

In [ ]:
male_80_plus = male_80_plus.rename(columns = {'NAME' : 'county_name',
                                              'P12_025N' : 'male pop 85+'})

# Placeholder for merging

In [ ]:
endpoint9 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_044N&for=county:*&in=state:47&key={0}'.format(api_key)

response9 = requests.get(endpoint9)

In [ ]:
female_65_66 = pd.DataFrame(response9.json())

In [ ]:
headers = female_65_66.iloc[0]
female_65_66 = female_65_66[1:]
female_65_66.columns = headers

In [ ]:
female_65_66 = female_65_66.rename(columns = {'NAME' : 'county_name',
                                              'P12_044N' : 'female pop 65-66'})

In [ ]:
endpoint10 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_045N&for=county:*&in=state:47&key={0}'.format(api_key)

response10 = requests.get(endpoint10)

In [ ]:
female_67_69 = pd.DataFrame(response10.json())

In [ ]:
headers = female_67_69.iloc[0]
female_67_69 = female_67_69[1:]
female_67_69.columns = headers

In [ ]:
female_67_69 = female_67_69.rename(columns = {'NAME' : 'county_name',
                                              'P12_045N' : 'female pop 67-69'})

In [ ]:
endpoint11 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_046N&for=county:*&in=state:47&key={0}'.format(api_key)

response11 = requests.get(endpoint11)

In [ ]:
female_70_74 = pd.DataFrame(response11.json())

In [ ]:
headers = female_70_74.iloc[0]
female_70_74 = female_70_74[1:]
female_70_74.columns = headers

In [ ]:
female_70_74 = female_70_74.rename(columns = {'NAME' : 'county_name',
                                              'P12_046N' : 'female pop 70-74'})

In [ ]:
endpoint12 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_047N&for=county:*&in=state:47&key={0}'.format(api_key)

response12 = requests.get(endpoint12)

In [ ]:
female_75_79 = pd.DataFrame(response12.json())

In [ ]:
headers = female_75_79.iloc[0]
female_75_79 = female_75_79[1:]
female_75_79.columns = headers

In [ ]:
female_75_79 = female_75_79.rename(columns = {'NAME' : 'county_name',
                                              'P12_047N' : 'female pop 75-79'})

In [ ]:
endpoint13 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_048N&for=county:*&in=state:47&key={0}'.format(api_key)

response13 = requests.get(endpoint13)

In [ ]:
female_80_84 = pd.DataFrame(response13.json())

In [ ]:
headers = female_80_84.iloc[0]
female_80_84 = female_80_84[1:]
female_80_84.columns = headers

In [ ]:
female_80_84 = female_80_84.rename(columns = {'NAME' : 'county_name',
                                              'P12_048N' : 'female pop 80-84'})

In [ ]:
endpoint14 = 'https://api.census.gov/data/2020/dec/dhc?get=NAME,P12_048N&for=county:*&in=state:47&key={0}'.format(api_key)

response14 = requests.get(endpoint14)

In [ ]:
female_85_plus = pd.DataFrame(response14.json())

In [ ]:
headers = female_85_plus.iloc[0]
female_85_plus = female_85_plus[1:]
female_85_plus.columns = headers

In [ ]:
female_85_plus = female_85_plus.rename(columns = {'NAME' : 'county_name',
                                                  'P12_048N' : 'female pop 85+'})

# Placeholder for merging